In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time
import os

In [2]:
def read_test_data(path):
    start_time = time.time()
    print("Start Read Test Data")
    data = np.load(path)
    print("Test data read --- %s seconds ---" % (time.time() - start_time))
    print(data)
    X_train = data["X_test"] # TODO
    Y_train = data["Y_test"]
    print("Testing - Total examples per class", np.sum(Y_train, axis=0))
    return [X_train, Y_train]

X_test, Y_test = read_test_data("/Users/blazejmanczak/Desktop/School/Year3/BEP/EvaluationRaf/testDataProcessedOriginal.npz")

Start Read Test Data
Test data read --- 0.029819965362548828 seconds ---
Testing - Total examples per class [ 680. 1185.  478.  329.   74.  160.  161.]


In [17]:
senet = keras.models.load_model("/Users/blazejmanczak/Desktop/senetRafFromAcc64OrgDataAcc877")

In [31]:
vgg = keras.models.load_model("/Users/blazejmanczak/Desktop/School/Year3/BEP/realTime/vggFaceAffectFullAdamFineTunedAcc64")

In [7]:
senet.evaluate(X_test, Y_test)

3067/3067 [==============================] - 219s 71ms/sample - loss: 1.0595 - accuracy: 0.8063


[1.0594631810072854, 0.8063254]

In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface import utils

def image_preprocessing(img):
    pixels = img*255
    #pixels = clahe((img*255).astype(np.uint8))
    pixels_expanded = np.expand_dims(pixels.astype(np.float64), axis=0)
    pre_pro = utils.preprocess_input(pixels_expanded, version=1)  # version 1 for VGG face
    return pre_pro[0]/255


datagenVal = ImageDataGenerator(
        preprocessing_function = image_preprocessing,
        dtype="float16")

Using TensorFlow backend.


In [19]:
evalDone = senet.evaluate(datagenVal.flow(X_test, Y_test))
print(evalDone)

  ...
    to  
  ['...']
96/96 [==============================] - 310s 3s/step - loss: 0.6734 - accuracy: 0.8781
[0.6734348892690226, 0.8780567]


In [27]:
predictionsSenet = senet.predict(datagenVal.flow(X_test))

# Quantizing the model

In [3]:
vggRaf = keras.models.load_model("/Users/blazejmanczak/Desktop/vggFaceFromAcc64OrgDataAcc853")

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(vggRaf)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_vggRaf = converter.convert()
f = open("vggRaf.tflite", "wb")
f.write(quantized_tflite_vggRaf)
f.close()

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(senet)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_senet = converter.convert()
f = open("senetRight.tflite", "wb")
f.write(quantized_tflite_senet)
f.close()

In [16]:
# A helper function to evaluate the TF Lite model using "validation" dataset.
# Referred from: https://www.tensorflow.org/lite/performance/post_training_integer_quant
def evaluate_model(interpreter):
    accurate_count = 0

    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    
    print(input_index)
    print(output_index)
    # Run predictions on every image in the "test" dataset.
    predictions = []
    for val_image, val_label in zip(X_test[:5], Y_test[:5]):
        val_image = image_preprocessing(val_image).astype(np.float32)
        val_image = tf.expand_dims(val_image, 0) 
        print(val_image)
        interpreter.set_tensor(input_index, val_image)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        probability = interpreter.get_tensor(output_index)
        print(probability)
        flower_id = np.argmax(probability[0])
        print(flower_id)
        predictions.append(flower_id)
        val_label = np.argmax(val_label)
        print(val_label)
       
        # Compare prediction results with ground truth labels to calculate accuracy.
        if flower_id == val_label:
            #print("right")
            accurate_count += 1
    
    accuracy = accurate_count * 1.0 / len(predictions)

    return accuracy

In [17]:
# Load the model into interpreters
interpreter_qat = tf.lite.Interpreter(model_path="vggRaf.tflite")
interpreter_qat.allocate_tensors()

# Evaluate the performance
print(evaluate_model(interpreter_qat))

80
0
tf.Tensor(
[[[[-0.3435059  -0.36377412 -0.46347567]
   [-0.3552706  -0.3794604  -0.48308352]
   [-0.3670353  -0.3912251  -0.49092665]
   ...
   [ 0.63296473  0.58916706  0.49338707]
   [ 0.63296473  0.58916706  0.49338707]
   [ 0.63296473  0.58916706  0.49338707]]

  [[-0.3435059  -0.3676957  -0.46739724]
   [-0.3552706  -0.3794604  -0.47916195]
   [-0.35919216 -0.38338196 -0.48308352]
   ...
   [ 0.63296473  0.58916706  0.49338707]
   [ 0.63296473  0.58916706  0.49338707]
   [ 0.63296473  0.58916706  0.49338707]]

  [[-0.34742746 -0.37161726 -0.4713188 ]
   [-0.35134903 -0.37553883 -0.47524038]
   [-0.34742746 -0.37161726 -0.4713188 ]
   ...
   [ 0.63296473  0.58916706  0.49338707]
   [ 0.63296473  0.58916706  0.49338707]
   [ 0.63296473  0.58916706  0.49338707]]

  ...

  [[ 0.63296473  0.58916706  0.49338707]
   [ 0.63296473  0.58916706  0.49338707]
   [ 0.63296473  0.58916706  0.49338707]
   ...
   [ 0.63296473  0.58916706  0.49338707]
   [ 0.63296473  0.58916706  0.49338707]


In [19]:
# Load the model into interpreters
interpreter_qat = tf.lite.Interpreter(model_path="senetRight.tflite")
interpreter_qat.allocate_tensors()

# Evaluate the performance
print(evaluate_model(interpreter_qat))

0.8764264753831106


In [ ]:
images = tf.cast(X_train, tf.float32)
emotion_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)

def representative_data_gen():
  for input_value in emotion_ds.take(3500):
    yield [input_value]